In [15]:
import json
import pandas as pd
import plotly.express as px

import numpy as np

import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
from flask_babel import gettext
from plotly.subplots import make_subplots
from scipy import stats
import numpy as np
import math
df_prov_tot = pd.read_csv('static/csv/be-covid-provinces-all.csv')
from datetime import datetime, date

#print(df_prov_tot)

# New hospitalizations in each province

In [16]:
def hospi_prov(prov=None):
    df_hospi = df_prov_tot

    if prov is not 'All':
        df_hospi = df_hospi[df_hospi['PROVINCE'] == prov]

    df = df_hospi.groupby(['DATE']).agg({'TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    x2 = df.index
    y2 = df.NEW_IN

    df.index = pd.to_datetime(df.index)
    dw = df.index.dayofweek
    colors_map = {0: '#fccde5', 1: '#8dd3c7', 2: '#b3de69', 3: '#bebada', 4: '#fb8072', 5: '#fdb462', 6: '#80b1d3',
                  7: 'lightgrey'}
    colors = [colors_map[i] for i in dw]

    
    fig = go.Figure(data=[
        go.Bar(x=x2[-30:], y=y2[-30:], name="bars", 
               legendgroup='group2', showlegend=True, marker_color=colors),
        go.Scatter(x=x2, y=y2.rolling(7,center=True).mean(), 
                   showlegend=True, name=gettext('avg'),legendgroup='avg 7 days', marker_color="red")
    ])
    start = '2021-01-01' 
    fig.update_layout(yaxis_range=[0,max(y2[x2 >=start])])
    fig.update_layout(xaxis_range=[start, datetime.today().strftime('%Y-%m-%d')])
    fig.update_layout(template="plotly_white", height=300, margin=dict(l=0, r=0, t=30, b=0),title=prov)

    return fig


def hospi_new_in_per_provinces():
    prov = ['All','Liège', 'Brussels','Antwerpen','Namur', 'Luxembourg', 'Hainaut' , 'BrabantWallon', 'VlaamsBrabant', 'OostVlaanderen',
        'WestVlaanderen', 'Limburg',]
    n_cols = 2
    n_rows = math.ceil(1.0 * len(prov)/ n_cols)

    idx = 0
    for prov_name in prov:
            fig = hospi_prov(prov=prov_name)
            
            fig.show()



hospi_new_in_per_provinces()

# Total hospitalizations in BE

In [17]:
def bar_hospitalization_tot():
    """
    bar plot hospitalization
    """
    df = df_prov_tot.groupby(['DATE']).agg({'TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    totin_bar = go.Scatter(x=df.index, y=df.TOTAL_IN, name='#Tot',stackgroup='two',line=dict(width=0.5),mode='lines', groupnorm="")

    fig_hospi = make_subplots(specs=[[{"secondary_y": True, }]], shared_yaxes='all', shared_xaxes='all')

    fig_hospi.add_trace(totin_bar, secondary_y=False)

    xvals = df.index[7:]
    yvals = 100 * (df.TOTAL_IN.values[7:] - df.TOTAL_IN.values[:-7]) / df.TOTAL_IN.values[:-7]

    fig_hospi.add_trace(go.Scatter(x=xvals[10:], y=yvals[10:], name='Week%',marker_color = 'pink'), secondary_y=True)

    fig_hospi.update_layout(template="plotly_white", height=500, margin=dict(l=0, r=0, t=30, b=0),
                            title="Total Hospitalizations")

    fig_hospi.update_layout(xaxis_title='Day',
                            yaxis_title='Number of / Day')

    fig_hospi.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig_hospi

bar_hospitalization_tot()

# Total ICU in BE

In [18]:
def bar_icu_tot():

    df = df_prov_tot.groupby(['DATE']).agg({'TOTAL_IN': 'sum', 'NEW_OUT': 'sum', 'NEW_IN': 'sum', 'TOTAL_IN_ICU': 'sum'})

    totin_bar = go.Scatter(x=df.index, y=df.TOTAL_IN_ICU, name='#Tot',stackgroup='two',line=dict(width=0.5),mode='lines', groupnorm="")

    fig_hospi = make_subplots(specs=[[{"secondary_y": True, }]], shared_yaxes='all', shared_xaxes='all')

    fig_hospi.add_trace(totin_bar, secondary_y=False)

    xvals = df.index[7:]
    yvals = 100 * (df.TOTAL_IN_ICU.values[7:] - df.TOTAL_IN_ICU.values[:-7]) / df.TOTAL_IN_ICU.values[:-7]

    fig_hospi.add_trace(go.Scatter(x=xvals[10:], y=yvals[10:], name='Week%',marker_color = 'pink'), secondary_y=True)

    fig_hospi.update_layout(template="plotly_white", height=500, margin=dict(l=0, r=0, t=30, b=0),
                            title="ICU")

    fig_hospi.update_layout(xaxis_title='Day',
                            yaxis_title='Number of / Day')

    fig_hospi.update_layout(
        hovermode='x unified',
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=list([
                    dict(
                        args=[{"yaxis.type": "linear"}],
                        label="LINEAR",
                        method="relayout"
                    ),
                    dict(
                        args=[{"yaxis.type": "log"}],
                        label="LOG",
                        method="relayout"
                    )
                ]),
            ),
        ])

    return fig_hospi

bar_icu_tot()